In [ ]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

**Load CIFAR10 Dataset**

In [ ]:
# Dowload the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

# Extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

  0%|          | 0/135107811 [00:00<?, ?it/s]

In [ ]:
data_dir = './data/cifar10'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'test']
['ship', 'automobile', 'airplane', 'deer', 'dog', 'bird', 'horse', 'frog', 'truck', 'cat']


In [ ]:
automobile_images = os.listdir(data_dir + "/train/automobile")
print('No. of training examples for automobiles:', len(automobile_images))
print(automobile_images[:5])

No. of training examples for automobiles: 5000
['1912.png', '3350.png', '1947.png', '1384.png', '1592.png']


In [ ]:
dataset = ImageFolder(data_dir+'/train', transform=ToTensor())
print('Length of the dataset:', len(dataset))

Length of the dataset: 50000


In [ ]:
def show_image(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))

In [ ]:
show_image(*dataset[100])

NameError: name 'show_example' is not defined

In [ ]:
show_example(*dataset[50])

**Preapre datasets for trainning and validation**

In [ ]:
# Set a random seed to always use the same trainning subset
random_seed = 51
torch.manual_seed(random_seed);

In [ ]:
val_size = 5000
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])
len(train_ds), len(val_ds)

In [ ]:
batch_size=128

train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)

In [ ]:
def show_imagebatch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=16).permute(1, 2, 0))
        break

In [ ]:
show_imagebatch(train_dl)

**Define CNN**

In [ ]:
def apply_kernel(image, kernel):
    ri, ci = image.shape       # image dimensions
    rk, ck = kernel.shape      # kernel dimensions
    ro, co = ri-rk+1, ci-ck+1  # output dimensions
    output = torch.zeros([ro, co])
    for i in range(ro): 
        for j in range(co):
            output[i,j] = torch.sum(image[i:i+rk,j:j+ck] * kernel)
    return output

In [ ]:
class ClassifyImages(nn.Module):
    def train(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        acc = accuracy(out, labels)         # Calculate accuracy
        return loss, acc
    
    def validate(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['train_acc'], result['val_loss'], result['val_acc']))
        
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
class Cifar10CnnModel0(ClassifyImages):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            #Conv Layer1: channels 3 to 32
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            #Conv Layer2: channels 32 to 64
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 64 x 16 x 16

            #Conv Layer3: channels 64 to 128
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            #Conv Layer4: channels 128 to 128
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 128 x 8 x 8

            #Conv Layer5: channels 128 to 256
            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            #Conv Layer6: channels 256 to 256
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2), # output: 256 x 4 x 4

            nn.Flatten(), 
            #Linear Layer1: 256*4*4 to 1024
            nn.Linear(256*4*4, 1024),
            nn.ReLU(),
            #Linear Layer2: 1024 to 512
            nn.Linear(1024, 512),
            nn.ReLU(),
            #Linear Layer3: 512 to 10
            nn.Linear(512, 10))
        
    def forward(self, xb):
        return self.network(xb)

In [ ]:
model0 = Cifar10CnnModel0()
model0

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
count_parameters(model0)

**Use GPU for trainning**

In [ ]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [ ]:
device = get_default_device()
device

**Place dataloaders and the model into GPU**

In [ ]:
train_dl = DeviceDataLoader(train_dl, device)
val_dl = DeviceDataLoader(val_dl, device)
to_device(model, device);

**Model Trainning Phase**

In [ ]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validate(batch) for batch in val_loader]
    return model.validation_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.Adam):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Trainning Phase 
        model.train()
        train_losses = []
        train_accuracies = []
        for batch in train_loader:
            loss, acc = model.train(batch)
            train_losses.append(loss)
            train_accuracies.append(acc)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['train_acc'] = torch.stack(train_accuracies).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [ ]:
def plot_accuracies(history):
    accuracies = [x['val_acc'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');
    
def plot_losses(history):
    train_losses = [x.get('train_loss') for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx')
    plt.plot(val_losses, '-rx')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs');

In [ ]:
#Sanity checking
evaluate(model0, val_dl)

In [ ]:
num_epochs = 20
lr = 0.001
opt_func = torch.optim.Adam

In [ ]:
#Start tranning
history0 = fit(num_epochs, lr, model0, train_dl, val_dl, opt_func)

In [ ]:
plot_losses(history0)

In [ ]:
plot_accuracies(history0)

In [ ]:
train_losses0 = [x['train_loss'] for x in history]
train_accuracies0 = [x['train_acc'] for x in history]
val_accuracies0 = [x['val_acc'] for x in history]

**Save the trained model, trainning losses and accuracies**

In [ ]:
torch.save(model0.state_dict(), 'HW_1_2_CIFAR10_model0.pth')
pd.DataFrame(train_losses0).to_csv('HW_1_2_CIFAR10_model0_train_losses.csv',index=False)
pd.DataFrame(train_accuracies0).to_csv('HW_1_2_CIFAR10_model0_train_accuracies.csv',index=False)
pd.DataFrame(val_accuracies0).to_csv('HW_1_2_CIFAR10_model0_val_accuracies.csv',index=False)